# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [56]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,al qaryatayn,34.2296,37.2407,16.53,48,100,4.90,SY,1712608012
1,1,polyarnyy,69.1989,33.4478,-3.72,93,100,6.86,RU,1712607920
2,2,invercargill,-46.4000,168.3500,16.12,74,100,2.68,NZ,1712607776
3,3,melekeok village,7.4957,134.6367,27.43,76,97,5.67,PW,1712608012
4,4,port-aux-francais,-49.3500,70.2167,7.54,83,79,15.20,TF,1712607925
5,5,saldanha,-33.0117,17.9442,16.03,90,95,10.82,ZA,1712607942
6,6,albany,42.6001,-73.9662,17.67,34,100,0.45,US,1712607583
7,7,n'zeto,-7.2312,12.8666,26.26,85,100,1.90,AO,1712608012
8,8,waitangi,-43.9535,-176.5597,14.34,89,100,2.68,NZ,1712607913
9,9,adamstown,-25.0660,-130.1015,23.78,84,73,2.62,PN,1712607922


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = "OSM",
    color='City',
    s='Humidity',
    cmap='tab20b'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
min_temperature = 21
max_temperature = 27
max_wind_speed = 1

city_criteria = (
    (city_data_df['Max Temp'] >= min_temperature) &
    (city_data_df['Max Temp'] < max_temperature) &
    (city_data_df['Wind Speed'] <= max_wind_speed)
)

# Drop any rows with null values
perfect_cities_df = city_data_df[city_criteria].dropna()  

# Display sample data
perfect_cities_df   

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,manokwari,-0.8667,134.0833,25.56,85,100,0.79,ID,1712608019
88,88,kuala belait,4.5833,114.1833,25.09,91,85,0.88,BN,1712608020
105,105,ilebo,-4.3167,20.5833,22.78,94,81,0.61,CD,1712608022
148,148,franceville,-1.6333,13.5836,22.15,97,100,0.62,GA,1712608027
248,248,baturaja,-4.1333,104.1667,23.73,99,100,0.99,ID,1712608038
316,316,daru,-9.0763,143.2092,26.78,85,100,0.93,PG,1712608026
447,447,kimbe,-5.5502,150.1429,24.92,95,100,0.42,PG,1712608061
448,448,lubao,14.9167,120.5500,25.84,78,100,0.64,PH,1712608061
479,479,zemio,5.0314,25.1361,23.10,66,11,0.82,CF,1712608064
483,483,kapit,2.0167,112.9333,23.84,99,98,0.71,MY,1712607923


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
81,manokwari,ID,-0.8667,134.0833,85,
88,kuala belait,BN,4.5833,114.1833,91,
105,ilebo,CD,-4.3167,20.5833,94,
148,franceville,GA,-1.6333,13.5836,97,
248,baturaja,ID,-4.1333,104.1667,99,
316,daru,PG,-9.0763,143.2092,85,
447,kimbe,PG,-5.5502,150.1429,95,
448,lubao,PH,14.9167,120.5500,78,
479,zemio,CF,5.0314,25.1361,66,
483,kapit,MY,2.0167,112.9333,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
manokwari - nearest hotel: Hotel Arfak
kuala belait - nearest hotel: Hotel Sentosa
ilebo - nearest hotel: Hôtel des Palmes
franceville - nearest hotel: Hotel Masuku
baturaja - nearest hotel: No hotel found
daru - nearest hotel: Kuki
kimbe - nearest hotel: Kimbe Bay Hotel
lubao - nearest hotel: Hacienda de Gracia Resort
zemio - nearest hotel: No hotel found
kapit - nearest hotel: Haip
langsa - nearest hotel: Harmoni Hotel
ca mau - nearest hotel: Khách sạn Song Ngọc
mutuali - nearest hotel: No hotel found
bukama - nearest hotel: Mission catholique


,City,Country,Lat,Lng,Humidity,Hotel Name
81,manokwari,ID,-0.8667,134.0833,85,Hotel Arfak
88,kuala belait,BN,4.5833,114.1833,91,Hotel Sentosa
105,ilebo,CD,-4.3167,20.5833,94,Hôtel des Palmes
148,franceville,GA,-1.6333,13.5836,97,Hotel Masuku
248,baturaja,ID,-4.1333,104.1667,99,No hotel found
316,daru,PG,-9.0763,143.2092,85,Kuki
447,kimbe,PG,-5.5502,150.1429,95,Kimbe Bay Hotel
448,lubao,PH,14.9167,120.5500,78,Hacienda de Gracia Resort
479,zemio,CF,5.0314,25.1361,66,No hotel found
483,kapit,MY,2.0167,112.9333,99,Haip


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    color='City',
    s='Humidity',
    cmap='tab20b',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],
    frame_width = 1000,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)